In [1]:
import pandas as pd
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer
import string
import spacy
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import BatchNormalization
from sklearn.linear_model import LogisticRegression

2024-06-08 11:43:30.087262: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-08 11:43:30.087379: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-08 11:43:30.249924: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
#test set
test = pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/test_essays.csv')
#submission format
sub = pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/sample_submission.csv')
#import data train
org_train = pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/train_essays.csv')
train = pd.read_csv("/kaggle/input/datasetllm/train_v2_drcat_02.csv", sep=',')

In [3]:
nlp = spacy.load("en_core_web_sm")
def clean_essay_spacy(essay):
    # Remove punctuation and non-alphanumeric characters
    essay = ''.join([char for char in essay if char not in string.punctuation and not char.isdigit()])

    # Convert to lowercase and process with spaCy
    doc = nlp(essay.lower())

    # Lemmatization and removing stopwords
    lemmatized_words = [token.lemma_ for token in doc if token.is_alpha and not token.is_stop]

    # Remove extra spaces
    cleaned_essay = ' '.join(lemmatized_words).strip()

    return cleaned_essay

In [4]:
cleaned_train_essays = []
cleaned_test_essays = []
cleaned_org_train_essays = []

train['cleaned_essay'] = train['text'].apply(clean_essay_spacy)
cleaned_train_essays.extend(train['cleaned_essay'].tolist())

test['cleaned_essay'] = test['text'].apply(clean_essay_spacy)
cleaned_test_essays.extend(test['cleaned_essay'].tolist())

org_train['cleaned_essay'] = org_train['text'].apply(clean_essay_spacy)
cleaned_org_train_essays.extend(org_train['cleaned_essay'].tolist())

# Build vocabulary of unique words from all cleaned essays
all_cleaned_essays = cleaned_train_essays + cleaned_test_essays + cleaned_org_train_essays
vocabulary = set(word for essay in all_cleaned_essays for word in essay.split())

# Print the size of the vocabulary
print(f"Vocabulary size: {len(vocabulary)}")

Vocabulary size: 77604


In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize the TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 3))

# Fit and transform the documents
tfidf_matrix = tfidf_vectorizer.fit_transform(all_cleaned_essays)

# Now, if you need to transform the documents separately after fitting to the whole corpus
train_tfidf = tfidf_vectorizer.transform(cleaned_train_essays)
test_tfidf = tfidf_vectorizer.transform(cleaned_test_essays)
org_train_tfidf = tfidf_vectorizer.transform(cleaned_org_train_essays)

# Print the shape of the matrices
print("Train TF-IDF shape:", train_tfidf.shape)
print("Test TF-IDF shape:", test_tfidf.shape)
print("Original Train TF-IDF shape:", org_train_tfidf.shape)

Train TF-IDF shape: (44868, 5840641)
Test TF-IDF shape: (3, 5840641)
Original Train TF-IDF shape: (1378, 5840641)


In [6]:
from sklearn.ensemble import RandomForestClassifier


y_train = train['label'].values

# Initialize the Random Forest classifier
model = RandomForestClassifier(n_estimators=100, random_state=42)

model.fit(train_tfidf, y_train)

RandomForestClassifier(random_state=42)

In [7]:
import pandas as pd

# Predict on the test set:
test_predictions = model.predict_proba(test_tfidf)[:, 1]


output_df = pd.DataFrame({
    'id': test['id'],  # Ensure this matches the column name for the IDs in your test DataFrame
    'probability': test_predictions  # This will be your model's predictions
})

# Save the updated DataFrame to a CSV file, ready for submission
output_df.to_csv('submission.csv', index=False)